In [17]:
drkg_file = './data/v1_res/relation/DDires.csv'

In [23]:
import pandas as pd
import numpy as np

df = pd.read_csv(drkg_file)
df = df.filter(['Drug', 'Palliates_Hetionet','Disease'], axis = 1)
triples = df.values.tolist()

In [24]:
df.head()

,Drug,Palliates_Hetionet,Disease
0,DrugBank:DB00997,0,DOID:363
1,DrugBank:DB00206,0,DOID:10763
2,DrugBank:DB00960,0,DOID:10763
3,DrugBank:DB00665,0,DOID:10283
4,DrugBank:DB00290,0,DOID:2998


In [25]:
num_triples = len(triples)
num_triples

2684037

In [26]:
seed = np.arange(num_triples)
np.random.shuffle(seed)

train_cnt = int(num_triples * 0.9)
valid_cnt = int(num_triples * 0.05)
train_set = seed[:train_cnt]
train_set = train_set.tolist()
valid_set = seed[train_cnt:train_cnt+valid_cnt].tolist()
test_set = seed[train_cnt+valid_cnt:].tolist()

In [27]:
with open("train/drkg_train.tsv", 'w+') as f:
    for idx in train_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))
        
with open("train/drkg_valid.tsv", 'w+') as f:
    for idx in valid_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))

with open("train/drkg_test.tsv", 'w+') as f:
    for idx in test_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))

In [28]:
!DGLBACKEND=pytorch dglke_train --dataset WANGKG --data_path ./train \
                               --data_files drkg_train.tsv drkg_valid.tsv drkg_test.tsv \
                               --format 'raw_udd_hrt' --model_name TransE_l2 --batch_size 2048 \
                               --neg_sample_size 256 --hidden_dim 400 --gamma 12.0 --lr 0.1 \
                               --max_step 100000 --log_interval 1000 --batch_size_eval 16 -adv \
                               --regularization_coef 1.00E-07 --test --num_thread 1 \
                               --gpu 0 1 2 3 4 5 6 7 --num_proc 8 \
                               --neg_sample_size_eval 10000 --async_update

Using backend: pytorch
Reading train triples....
Finished. Read 2415633 train triples.
Reading valid triples....
Finished. Read 134201 valid triples.
Reading test triples....
Finished. Read 134203 test triples.
|Train|: 2415633
random partition 2415633 edges into 8 parts
part 0 has 301955 edges
part 1 has 301955 edges
part 2 has 301955 edges
part 3 has 301955 edges
part 4 has 301955 edges
part 5 has 301955 edges
part 6 has 301955 edges
part 7 has 301948 edges
/home/ubuntu/anthony/KG_LinkPrediction_Project/my_project_env/lib/python3.8/site-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)
/home/ubuntu/anthony/KG_LinkPrediction_Project/my_project_env/lib/python3.8/site-packages/dgl/base.py:45: DGLWarning: Keyword arguments ['readonly', 'multigraph', 'sort_csr'] are deprecated in v0.5, and can be safely removed in all cases.
  return warnings.warn(message, ca